# Кибериммунная автономность$\\$Создание конструктивно защищённого автономного наземного транспортного средства$\\$Модуль 4

## О документе

Версия 1.03

Модуль 4 для регионального этапа соревнований по кибериммунной автономности

### Модуль 4. Следование по трассе с киберпрепятствиями

Для успешного выполнения этого задания необходимое активировать специальный режим работы виртуальной машинки - в рамках этого задания можно менять только блоки, отвечающие за безопасность (ограничитель, монитор безопасности), **другие блоки менять запрещено**.

При прохождении маршрута будут имитироваться атаки со стороны злоумышленников, которые будут пытаться нарушить цели безопасности. Важно, чтобы им это не удалось.

1. Запустите свою машинку и убедитесь, что она проходит всю трассу без нарушений ограничений скорости. При необходимости измените логику работы блока безопасности
2. Добавьте контроль доставки груза в модуле SafetyBlock - убедитесь, что груз доставляется до конечной точки маршрута.
3. Если в модуле 3 вы реализовали монитор безопасности - не забудьте его перенести в этот модуль, это принесёт дополнительные баллы!

Активация киберпрепятствий в системе управления:

после инициализации системы управления добавьте следующую строку
```python
control_system.enable_surprises()
```

В этом блоке добавьте все ваши реализации изменённых бортовых систем

In [5]:
# ваш код

# если вы собираетесь выполнить это задание, напишите в этой ячейке код выполнения изменённых блоков. 
# cам код допускается редактировать в папке src
# в случае успешной реализации машинка должна успешно пройти по маршруту, а в случае искажения маршрутного задания в блоке коммуникационного шлюза должна остаться на месте, 
# при этом именно блок безопасности должен заблокировать движение
from src.safety_block import BaseSafetyBlock
from src.config import *
import math
from time import sleep
from geopy import Point as GeoPoint
from src.config import LOG_ERROR, LOG_INFO, LOG_DEBUG
from multiprocessing import Queue
from src.config import SERVOS_QUEUE_NAME
from src.event_types import Event
from src.control_system import BaseControlSystem
from src.navigation_system import BaseNavigationSystem
from src.communication_gateway import BaseCommunicationGateway
from multiprocessing import Queue
from src.config import SAFETY_BLOCK_QUEUE_NAME
from src.config import SERVOS_QUEUE_NAME
from src.config import CONTROL_SYSTEM_QUEUE_NAME
from src.event_types import Event
from src.crypto import *
from src.mission_type import *
from src.route import *
from time import sleep
from geopy import Point as GeoPoint


from src.queues_dir import QueuesDirectory
from src.servos import Servos
from src.sitl import SITL
from src.cargo_bay import CargoBay
from src.mission_planner import MissionPlanner
from src.config import LOG_ERROR, LOG_INFO
from src.mission_planner_mqtt import MissionSender
from src.mission_planner import Mission
from src.sitl_mqtt import TelemetrySender
from src.system_wrapper import SystemComponentsContainer
from src.wpl_parser import WPLParser

# добавьте изменения сюда

from src.security_monitory import BaseSecurityMonitor
from src.security_policy_type import SecurityPolicy

from src.config import *
    
    
class SecurityMonitor(BaseSecurityMonitor):
    """ класс монитора безопасности """

    def __init__(self, queues_dir):
        super().__init__(queues_dir)
        self._init_set_security_policies()


    def _init_set_security_policies(self):
        """ инициализация политик безопасности """
        default_policies = [
            SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='set_mission'),
            SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation="set_mission"),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=NAVIGATION_QUEUE_NAME,
                operation="request_position"),
            SecurityPolicy(
                source=NAVIGATION_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation="position_update"),
            SecurityPolicy(
                source=NAVIGATION_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation="position_update"),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation="set_speed"),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation="set_direction"),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation="lock_cargo"),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation="release_cargo"),            
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=SERVOS_QUEUE_NAME,
                operation="set_speed"),
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=SERVOS_QUEUE_NAME,
                operation="set_direction"),
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=CARGO_BAY_QUEUE_NAME,
                operation="release_cargo"),
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=CARGO_BAY_QUEUE_NAME,
                operation="lock_cargo")
        ]
        self.set_security_policies(policies=default_policies)        

    def set_security_policies(self, policies):
        """ установка новых политик безопасности """
        self._security_policies = policies
        self._log_message(
            LOG_INFO, f"изменение политик безопасности: {policies}")

    def _check_event(self, event: Event):
        """ проверка входящих событий """
        self._log_message(
            LOG_DEBUG, f"проверка события {event}, по умолчанию выполнение запрещено")

        authorized = False
        request = SecurityPolicy(
            source=event.source,
            destination=event.destination,
            operation=event.operation)

        if request in self._security_policies:
            self._log_message(
                LOG_DEBUG, "событие разрешено политиками, выполняем")
            authorized = True

        if authorized is False:
            self._log_message(LOG_ERROR, f"событие не разрешено политиками безопасности! {event}")
        return authorized

class SafetyBlock(BaseSafetyBlock):
    """ класс ограничений безопасности """
    def _set_mission(self, signature, mission: Mission):
        """ установка нового маршрутного задания """
        if not self._verify_mission(mission, signature):
            self._log_message(LOG_ERROR, "попытка установки левого mission")
            return
        
        self._mission = mission
        self._route = Route(points=self._mission.waypoints,
                            speed_limits=self._mission.speed_limits)
        
    def _set_new_direction(self, signature, direction: float):
        """ установка нового направления перемещения """
        self._log_message(LOG_INFO, f"текущие координаты: {self._position}")
        self._log_message(LOG_DEBUG, f"маршрутное задание: {self._mission}")
        self._log_message(LOG_DEBUG, f"состояние маршруте: {self._route}")
        # TODO реализовать контроль безопасности изменения направления

        if not self._position or not self._route:
            self._log_message(LOG_ERROR, f"не получилось установить новое направление - неизвестный маршрут или позиция машинки")
            return
        
        current = self._position
        next = self._route.next_point()
        bearing = self._calculate_bearing(current, next)
        
        new_direction = max(bearing - 5, min(bearing + 5, direction))
        
        if new_direction != direction:
            self._log_message(LOG_ERROR, f"попытка изменить направление движения не в ту сторону: {direction}. Скорректировано на {new_direction}")
                
        self._direction = new_direction
        # отправка сообщения с желаемым направлением
        self._send_direction_to_consumers()

    def _set_new_speed(self, signature, speed: float):
        """ установка новой скорости """
        if not self._route:
            self._log_message(LOG_ERROR, f"не получилось установить новую скорость - неизвестный маршрут")
            return

        new_speed = max(0, min(speed, self._route.calculate_speed()))
        if new_speed != speed:
            self._log_message(LOG_ERROR, f"попытка изменить скорость движения на запрещенную: {speed}. Скорректировано на {new_speed}")
                
        self._speed = new_speed
        self._send_speed_to_consumers()

    def _send_speed_to_consumers(self):
        self._log_message(LOG_DEBUG, "отправляем скорость получателям")
        servos_q_name = SERVOS_QUEUE_NAME
        
        security_monitor_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)

        # отправка сообщения с желаемой скоростью
        event_speed = Event(source=self.event_source_name,
                            destination=servos_q_name,
                            operation="set_speed",
                            parameters=self._speed
                            )
        security_monitor_q.put(event_speed)

    def _send_direction_to_consumers(self):
        self._log_message(LOG_DEBUG, "отправляем направление получателям")

        servos_q_name = SERVOS_QUEUE_NAME
        security_monitor_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)

        # отправка сообщения с желаемой скоростью
        event_speed = Event(source=self.event_source_name,
                            destination=servos_q_name,
                            operation="set_direction",
                            parameters=self._direction
                            )
        security_monitor_q.put(event_speed)

    def _lock_cargo(self, signature, _):
        self._log_message(LOG_INFO, "Блокировка грузового отсека")
        self._send_lock_cargo_to_consumers()

    def _send_lock_cargo_to_consumers(self):
        self._log_message(LOG_DEBUG, "Отправляем команду блокировки в CargoBay")
        
        event = Event(
            source=self.event_source_name,
            destination=CARGO_BAY_QUEUE_NAME,
            operation="lock_cargo",
            parameters=None
        )
        security_monitor_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)
        
    def _release_cargo(self, signature, _):
        if not self._route or not self._route.route_finished:
            return
        self._send_release_cargo_to_consumers()

    def _send_release_cargo_to_consumers(self):
        self._log_message(LOG_DEBUG, "Отправляем команду выгрузки в CargoBay")
        event = Event(
            source=self.event_source_name,
            destination=CARGO_BAY_QUEUE_NAME,
            operation="release_cargo",
            parameters=None
        )
        security_monitor_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)

    def _calculate_bearing(self, start: GeoPoint, end: GeoPoint):
        delta_longitude = end.longitude - start.longitude
        x = math.sin(math.radians(delta_longitude)) * \
            math.cos(math.radians(end.latitude))
        y = math.cos(math.radians(start.latitude)) * math.sin(math.radians(end.latitude)) - \
            math.sin(math.radians(start.latitude)) * math.cos(math.radians(end.latitude)) * \
            math.cos(math.radians(delta_longitude))
        initial_bearing_rad = math.atan2(x, y)

        # Преобразуем радианы в градусы
        initial_bearing_deg = math.degrees(initial_bearing_rad)

        # Нормализуем значение в диапазоне [0, 360]
        compass_bearing = (initial_bearing_deg + 360) % 360

        return compass_bearing
    
    def _verify_mission(self, mission, signature):
        """
        Verify the signature of the route using the RSA public key.
        Returns True if the signature is valid, False otherwise.
        """
        data = serialize(mission)
        h_int = hash_data(data)
        n, e = self._public_key
        # Recover the hash from the signature: recovered = signature^e mod n
        recovered_hash = pow(signature, e, n)
        return h_int == recovered_hash

        
class CommunicationGateway(BaseCommunicationGateway):
    """CommunicationGateway класс для реализации логики взаимодействия
    с системой планирования заданий

    Работает в отдельном процессе, поэтому создаётся как наследник класса Process
    """
    def _send_mission_to_consumers(self):        
        """ метод для отправки сообщения с маршрутным заданием в систему управления """
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME
        signature = self._sign_mission()
        event = Event(source=BaseCommunicationGateway.event_source_name,
                      destination=control_q_name,
                      operation="set_mission", parameters=self._mission, signature=signature
                      )
        security_monitor_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)
                
        safety_q_name = SAFETY_BLOCK_QUEUE_NAME
        event = Event(source=BaseCommunicationGateway.event_source_name,
                      destination=safety_q_name,
                      operation="set_mission", parameters=self._mission, signature=signature
                      )
        security_monitor_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)

    def _sign_mission(self):
        """
        Sign the serialized route using the provided private key.
        Returns the digital signature as bytes.
        """
        data = serialize(self._mission)
        h_int = hash_data(data)
        n, d = self._private_key
        # RSA signature: signature = hash^d mod n
        signature = pow(h_int, d, n)
        return signature


class ControlSystem(BaseControlSystem):
    """ControlSystem блок расчёта управления """

    def _send_speed_and_direction_to_consumers(self, speed, direction):
        safety_q_name = SAFETY_BLOCK_QUEUE_NAME # замените на правильное название очереди

        # отправка сообщения с желаемой скоростью
        event_speed = Event(source=BaseControlSystem.event_source_name,
                      destination=safety_q_name,
                      operation="set_speed", parameters=speed
                    ) # замените на код создания сообщения со скоростью для приводов
                           # подсказка, требуемая операция - set_speed

        # отправка сообщения с желаемым направлением
        event_direction = Event(source=BaseControlSystem.event_source_name,
                      destination=safety_q_name,
                      operation="set_direction", parameters=direction
                    ) # замените на код создания сообщения с направлением для приводов
                               # подсказка, требуемая операция - set_direction

        security_monitor_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event_direction)
        security_monitor_q.put(event_speed)
        
    def _lock_cargo(self):
        """ заблокировать грузовой отсек """
        event = Event(source=CONTROL_SYSTEM_QUEUE_NAME,
                    destination=SAFETY_BLOCK_QUEUE_NAME,
                    operation="lock_cargo", parameters= None
                    )
        security_monitor_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)

    def _release_cargo(self):
        """ открыть грузовой отсек """
        event = Event(source=CONTROL_SYSTEM_QUEUE_NAME, 
                    destination=SAFETY_BLOCK_QUEUE_NAME,
                    operation="release_cargo", parameters= None
                    )  
        security_monitor_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)
        

class NavigationSystem(BaseNavigationSystem):
    """ класс навигационного блока """
    def _send_position_to_consumers(self):        
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME
        event = Event(source=BaseNavigationSystem.event_source_name,
                      destination=control_q_name,
                      operation="position_update", parameters=self._position
                      )
        security_monitor_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)
      
        safety_q_name = SAFETY_BLOCK_QUEUE_NAME # замените на правильное название очереди
        event =  Event(source=BaseNavigationSystem.event_source_name,
                      destination=safety_q_name,
                      operation="position_update", parameters=self._position
                    ) # замените на код создания сообщения с координатами для системы управления 
                     # подсказка, требуемая операция - position_update
        security_monitor_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)


Если у вас настроена и работает СУПА, установите в True значение переменной afcs_present

In [6]:
afcs_present = True

Поменяем идентификатор машинки для этого модуля

In [7]:
car_id = "m4"

В следующем блоке измените маршрут на ваш, его можно скопировать из модуля 2.
 
Для проверки работы систем безопасности будут активированы киберпрепятствия

```python
control_system.enable_critical_surprises()
```

In [8]:
# используем то же маршрутное задание, которое было в модуле 2
from time import sleep

from src.queues_dir import QueuesDirectory
from src.servos import Servos
from src.sitl import SITL
from src.cargo_bay import CargoBay
from src.mission_planner import MissionPlanner
from src.config import LOG_ERROR, LOG_INFO
from src.mission_planner_mqtt import MissionSender
from src.mission_planner import Mission
from src.sitl_mqtt import TelemetrySender
from src.system_wrapper import SystemComponentsContainer
from src.wpl_parser import WPLParser
from src.mission_type import GeoSpecificSpeedLimit


# возьмём маршрут из модуля 2
wpl_file = "module_2.wpl"

parser = WPLParser(wpl_file)    
points = parser.parse()

# обновите скоростные ограничения для вашего маршрута!
speed_limits = [
    GeoSpecificSpeedLimit(0, 20),
    GeoSpecificSpeedLimit(3, 50),
    GeoSpecificSpeedLimit(6, 20),
    GeoSpecificSpeedLimit(8, 40),
    GeoSpecificSpeedLimit(9, 20),
]


home = points[0]
mission = Mission(home=home, waypoints=points,speed_limits=speed_limits, armed=True)

# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory() 

private_key, public_key = generate_keys(1024)

# создание блоков передачи данных в СУПА
if afcs_present:
    mission_sender = MissionSender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)
    telemetry_sender = TelemetrySender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)

# создание основных функциональных блоков
mission_planner = MissionPlanner(
    queues_dir, afcs_present=afcs_present, mission=mission)

sitl = SITL(
    queues_dir=queues_dir, position=home,
    car_id=car_id, post_telemetry=afcs_present, log_level=LOG_ERROR)

communication_gateway = CommunicationGateway(private_key=private_key,
    queues_dir=queues_dir, log_level=LOG_ERROR)
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)

navigation_system = NavigationSystem(
    queues_dir=queues_dir, log_level=LOG_ERROR)

servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)

safety_block = SafetyBlock(public_key=public_key, queues_dir=queues_dir, log_level=LOG_INFO)

security_monitor = SecurityMonitor(queues_dir=queues_dir)

# сборка всех запускаемых блоков в одном "кузове"
system_components = SystemComponentsContainer(
    components=[
        mission_sender,
        telemetry_sender,
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block,
        security_monitor
    ] if afcs_present else [
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block,
        security_monitor
    ])

#################################
# АКТИВАЦИЯ КИБЕРПРЕПЯТСТВИЙ
control_system.enable_surprises()
#################################

# запуск всех блоков
system_components.start()

# ограничение поездки по времени
# в случае превышения времени выполнения ячейки на более чем 10 секунд от заданного, 
# допустимо перезапустить вычислительное ядро и повторно выполнить весь блокнот, штрафные очки за это не начисляются
# при условии, что повторный запуск закончился успешно
sleep(100)

# останавливаем все компоненты
system_components.stop()

# удалим все созданные компоненты
system_components.clean()

[ИНФО][QUEUES] создан каталог очередей
[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован
[ИНФО][QUEUES] регистрируем очередь safety
[ИНФО][SAFETY] создан ограничитель
[ИНФО][QUEUES] регистрируем очередь security
[ИНФО][SECURITY] создан монитор безопасности
[ИНФО][SECURITY] изменение политик безопасности: [SecurityPolicy(source='communication', destination='control', operation='set_mission'), SecurityPolicy(source='communication', destination='safety', operation='set_m

[ИНФО][MISSION PLANNER] старт системы планирования заданий
[ИНФО][CARGO] старт блока грузового отсека
[ИНФО][CONTROL] старт системы управления
[ИНФО][SAFETY] старт ограничителя
[ИНФО][SECURITY] старт блока грузового отсека
[ИНФО][MISSION PLANNER] запрошена новая задача, отправляем получателям
[ИНФО][MISSION PLANNER] новая задача отправлена в коммуникационный шлюз
[ИНФО][CONTROL] установлена новая задача, начинаем следовать по маршруту, текущее время 19:33:17.682162
[ИНФО][SAFETY] Блокировка грузового отсека
[ИНФО][CONTROL] новая скорость 20 (была 0)
[ИНФО][CONTROL] новое направление 294 (было 0)
[ИНФО][CARGO] заблокировать грузовой отсек
[ИНФО][CARGO] грузовой отсек заблокирован
[ИНФО][SAFETY] текущие координаты: 59 52m 53.4688s N, 29 49m 48.1732s E
[ИНФО][SAFETY] текущие координаты: 59 52m 53.4688s N, 29 49m 48.1732s E
[ИНФО][SAFETY] текущие координаты: 59 52m 53.4688s N, 29 49m 48.1732s E
[ИНФО][SAFETY] текущие координаты: 59 52m 53.4688s N, 29 49m 48.1732s E
[ИНФО][SAFETY] текущие к

Убедитесь, что 
1. ваша машинка успешно прошла весь заданный маршрут
2. не превысила ограничения скорости
3. успешно доставила груз

Если всё так - поздравляем, вы справились с заданием! Обязательно зафиксируйте все изменения в репозитории!



Мы будем признательны за обратную связь - любые комментарии, которые вы можете дать по итогам выполнения этого задания. 

Например, 

- было ли задание понятным по шкале 1..10 (1 - ничего не понятно, 10 - вопросов вообще не было, всё понятно)?
- было ли задание интересным по шкале 1..10 (1 - скука смертная, 10 - лучшее, что вам пока встречалось на олимпиадах)? 
- что бы вы предложили изменить, чтобы сделает его более интересным?
- по шкале 1..10 насколько сложным оно было для вас?
- что было самым трудным в задании? 

Авторы наиболее развёрнутых и интересных комментариев получат особенный приз от Лаборатории Касперского!

Дополнительная информация о кибериммунной разработке
- https://os.kaspersky.ru/cyber-immune-development/ 
- https://github.com/sergey-sobolev/cyberimmune-systems/wiki/%D0%9A%D0%B8%D0%B1%D0%B5%D1%80%D0%B8%D0%BC%D0%BC%D1%83%D0%BD%D0%B8%D1%82%D0%B5%D1%82
- канал в телеграм: https://t.me/learning_cyberimmunity
